# Counts FPKM RPKM TPM CPM 的转化

## 一、什么是RPKM、 FPKM、TPM、CPM
[RPKM, FPKM and TPM, clearly explained - StatQuest!!!](https://link.zhihu.com/?target=https%3A//statquest.org/rpkm-fpkm-and-tpm-clearly-explained/)

对基因counts进行校正定量一般有RPKM、FPKM 、TPM和CPM这几种方法，StatQuest网站中对RPKM, FPKM 和 TPM作了通俗简要的说明，现将其核心要点整理如下：



## RPKM

> RPKM (Reads Per Kilobase Million, or Reads Per Kilobase of transcript per Million reads mapped)
> 每千个碱基的转录每百万映射读取的reads

计算RPKM的方法：

- 计算样本中的总reads，并将该数字除以1,000,000——这是我们的“每百万”缩放因子 ( “per million” scaling factor ) 。
- 将reads数除以“每百万”缩放因子。消除测序深度影响，得到每百万reads（RPM, reads per million ）。
- 将RPM值除以基因长度（以千碱基为单位），消除基因长度影响，得到RPKM。
FPKM

## FPKM
> FPKM (Fragments Per Kilobase Million, or Fragments Per Kilobase of transcript per Million reads mapped) 每千个碱基的转录每百万映射读取的fragments

PKM与RPKM非常相似。RPKM是针对单端测序的RNA-seq而言的，其中每个reads对应于一个已测序的单个片段。FPKM用于双端测序的RNA-seq。使用双端测序RNA-seq，两个reads可以对应一个片段（Fragment）。RPKM和FPKM之间的唯一区别是FPKM考虑到两次reads可以映射到一个片段（因此它不会对该片段进行两次计数）。 即 **单端测序：reads=fragments，双端测序：2 * reads≈fragments**


经过上游处理，双端测序两个reads可以对应一个片段的过程已经完成，最后得到的counts就已经相当于是片段fragments了，因此下游分析由counts计算RPKM、 FPKM这两者的公式完全一致。


## TPM
> TPM (Transcripts Per Million, or Transcripts Per kilobase of exon model per Million mapped reads)
> 每千个碱基的转录每百万映射读取的Transcripts

计算TPM的方法：

- 将读数计数除以每个基因的长度（以千碱基为单位），得到每千碱基reads（RPK, reads per kilobase）。
- 计算样本中所有RPK值，然后将其除以1,000,000，得到“每百万”缩放因子 ( “per million”scaling factor )。
- 将RPK值除以“每百万”缩放因子，得到TPM。

因此在计算TPM时，与RPKM、 FPKM相比，唯一的区别是TPM先对基因长度进行标准化，然后对测序深度进行标准化。但是，这种差异的影响非常深远。因为使用TPM时，每个样本中所有TPM的总和是相同的，这样可以更轻松地比较每个样本中映射到基因的读数的比例。相反，使用RPKM和FPKM，每个样本中的标准化读数之和可能会有所不同，这使得直接比较样本变得更加困难。

## CPM
> CPM（Counts Per Million, or Counts of exon model Per Million mapped reads） 每百万映射读取的counts

除了`RPKM`、 `FPKM`、`TPM`这几种方法，`CPM`也是较为常见的一种基因定量方式。原始的表达量除以该样本表达量的总和，再乘以一百万，即可得到`CPM`值。`CPM`值只对测序深度进行了标准化，一般利用`edgeR`包的cpm()函数即可对基因`counts`进行简单校正 。

```R
edgeR::cpm(counts)
```


## 二、由Counts计算FPKM/RPKM和TPM
有许多文章已经给出了这几种计数方式的计算和转化关系，如[What the FPKM? A review of RNA-Seq expression units | The farrago (wordpress.com)。](https://link.zhihu.com/?target=https%3A//haroldpimentel.wordpress.com/2014/05/08/what-the-fpkm-a-review-rna-seq-expression-units/)

In [1]:
countToTpm <- function(counts, effLen) {
    rate <- log(counts) - log(effLen)
    denom <- log(sum(exp(rate)))
    exp(rate - denom + log(1e6))
}

countToFpkm <- function(counts, effLen) {
    N <- sum(counts)
    exp(log(counts) + log(1e9) - log(effLen) - log(N))
}

fpkmToTpm <- function(fpkm) {
    exp(log(fpkm) - log(sum(fpkm)) + log(1e6))
}

countToEffCounts <- function(counts, len, effLen) {
    counts * (len / effLen)
}

########## An example ##########
cnts <- c(4250, 3300, 200, 1750, 50, 0)
lens <- c(900, 1020, 2000, 770, 3000, 1777)
countDf <- data.frame(count = cnts, length = lens)

# assume a mean(FLD) = 203.7
countDf$effLength <- countDf$length - 203.7 + 1
countDf$tpm <- with(countDf, countToTpm(count, effLength))
countDf$fpkm <- with(countDf, countToFpkm(count, effLength))
with(countDf, all.equal(tpm, fpkmToTpm(fpkm)))
countDf$effCounts <- with(countDf, countToEffCounts(count, length, effLength))


[1] TRUE